<a href="https://colab.research.google.com/github/idrimadrid/Deep_learning_Segmentation_CAMUS_challenge/blob/main/camus_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install medpy
!pip -q install SimpleITK

     |████████████████████████████████| 153kB 1.8MB/s 
     |████████████████████████████████| 42.5MB 108kB/s 


In [ ]:
import tensorflow as tf
import numpy as np
import nibabel as nib
from glob import glob
from medpy.io import load, save
import SimpleITK as sitk
from skimage.transform import resize
from IPython.display import clear_output
# (0.308, 0.154, 1.54)

In [ ]:
header = None
for i in range(1, 10):
  file = f"/content/drive/My Drive/Camus Test/test/patient000{i}/patient000{i}_2CH_ED.mhd"
  image = load(file)
  header = image[1]
  break

In [ ]:
def normalize(input_image):
  """ normalisation [0..1] """
  input_image = tf.cast(input_image, tf.float32) / 255.0
  return input_image

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [ ]:
modelPath = "/content/drive/My Drive/camusModel.h5"
model = tf.keras.models.load_model(modelPath)

In [ ]:
input_folder = "/content/drive/My Drive/testing_converted/"
output_folder = "/content/drive/My Drive/predicted/"

In [ ]:
resize_shape = (352, 352, 1)
pat = {}
predictions = []

Example:

{
  
	"name" : "patient0001"

	"patient0001_2CH_ED.nii": array

	"patient0001_2CH_ES.nii": array

	"patient0001_4CH_ED.nii": array

	"patient0001_4CH_ES.nii": array

}

In [ ]:
for index, patient in enumerate(glob(input_folder+"/p*")):
  pat["name"] = patient[-11:]
  for file in glob(patient+"/*"):
    file_name = file[-22:]
    image = nib.load(file).get_fdata()
    normal_resized_image = normalize(resize(image, resize_shape))
    image_batched = normal_resized_image[tf.newaxis, ...]
    predict = model.predict(image_batched) 
    output = create_mask(predict).numpy()
    pat["original_size"] = image.shape
    pat[file_name] = np.ndarray.astype(resize(output, pat['original_size'], preserve_range=True), np.uint8)
  predictions.append(pat)
  pat = {}
  print(index*2, "%")
  clear_output(True)
print("Loading completed.")

Loading completed.


Save predictions to mhd format

In [ ]:
for index, predicted in enumerate(predictions):
	name = ""
	for k, v in predicted.items():
		if k == "name":
			name = v
		if "nii" in k:
			save(v, output_folder+"/"+k.replace("nii", "mhd"), hdr=header)
	print(index*100//len(predictions))
print("Done.")

0
Done.


In [ ]:
#conversion


# input_folder = "/content/drive/My Drive/Camus Test/test"
# converted_folder = "/content/drive/My Drive/testing_converted/"

# for index, patient in enumerate(glob(input_folder+"/p*")):
#   folder_name = patient[-11:]
#   for file in glob(patient+"/*.mhd"):
#     converted_file_name = file[52:].replace("mhd", "nii")
#     Path(converted_folder+folder_name).mkdir(parents=True, exist_ok=True)
#     if "sequence" not in file:
#       img = sitk.ReadImage(file)
#       sitk.WriteImage(img, converted_folder+"/"+folder_name+"/"+converted_file_name)
#   clear_output(True)
#   print(index)